In [143]:
%load_ext autoreload
%autoreload 2

from utils import *
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

from torch.utils import data

import glob
import random
import os
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

from bokeh.io import push_notebook, show, output_notebook,reset_output
from bokeh.plotting import figure
import networkx as nx
from bokeh.plotting import figure, from_networkx, curdoc
from bokeh.models import Rect, HoverTool,  TextInput, GraphRenderer, Circle
from bokeh.io import output_file
from bokeh.plotting import figure, output_file, save
from bokeh.models import (BoxZoomTool, Circle, Line, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool)
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar,CustomJSHover,CustomJS
from bokeh.models import Ellipse, GraphRenderer, StaticLayoutProvider

import grandalf

output_notebook()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
QDIR = os.path.join("/Users/pari/LatencyCollector/queries")
JOB_QNAMES = get_qnames(os.path.join(QDIR, "job"))
CEB_QNAMES = get_qnames(os.path.join(QDIR, "ceb-small"))
ERG_QNAMES = get_qnames(os.path.join(QDIR, "ergast-small"))
STATS_QNAMES = get_qnames(os.path.join(QDIR, "stats"))
JOBLT_QNAMES = get_qnames(os.path.join(QDIR, "joblight_train"))
STACK_QNAMES = get_qnames(os.path.join(QDIR, "stack-small"))
TPCH_QNAMES = get_qnames(os.path.join(QDIR, "tpch"))
ACC_QNAMES = get_qnames(os.path.join(QDIR, "accidents"))

LT_FN = "lt_instances.txt"

#EXP_NAMES = ["exp14_single_stack"]

#EXP_NAMES = ["exp1"]

#EXP_NAMES = ["exp6_multi4b"]

EXP_NAMES = ["exp1_single", "exp2_single", "exp3_single", "exp4_single_ergast", "exp5_single_stats",
            "exp6_single_joblight", "exp7_single_ceb2", "exp8_single_stack", "exp9_single_tpch", 
             "exp10_single_zdbs", "exp11_single_zdbs", "exp14_single_stack",
             "exp11-ceb-tpch-ergast-mixed-n3", 
             "exp7-stack-mixed-n3",
             "exp8-stack-imdb-mixed-n3",
             "exp8-stack-mixed-n6",
             "exp13-zdbs-mt-n2", "exp13-zdbs-mt-n4", "exp13-zdbs-mt-n8",
            ]

EXP_NAMES = ["exp1_single", "exp2_single", "exp3_single", "exp13_imdb"]

#EXP_NAMES = ["exp13_imdb"]
#RESULTDIRS = ["new"]
RESULTDIRS = ["new", "concurrent"]


#EXP_NAMES = ["exp1-imdb-n2"]
#EXP_NAMES = ["exp2-imdb-n8"]

#EXP_NAMES = [ "exp10_single_zdbs"]

#EXP_NAMES = ["exp5_single_stats"]
#EXP_NAMES = ["exp13_imdb"]
#EXP_NAMES = ["exp9_single_tpch"]

#RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "new", EXP_NAME)

LT_TYPES = '''a1_large_mag_4g=lt-0d15fb8f5bbe9a27d
a1_large_gp3_4g=lt-04840b55d3f795395
r7g_large_gp2_16g=lt-0212ec953ba35b176
t3_large_gp2_8g=lt-05d2d354bc3dd9133
c5a_large_mag_4g=lt-03218e9e27718bbbe
m6a_large_mag_8g=lt-0f6f46002652f9a4c
t3a_medium_gp3_4g=lt-0af65294350b1a8c1
t3a_large_gp3_8g=lt-084bfbae110d52d4e
r6a_large_mag_16g=lt-0e608666ff3adff07
t4g_large_mag_8g=lt-04e0b4826c63bfadb
c7g_large_mag_4g=lt-0af47c6caa3b53b8b
r7g_medium_gp2_16g=lt-01d0081183a7d79f2
t3_xlarge_gp2_16g=lt-0b413bcc22b3ac8fb
'''

lt_types = pd.read_csv(StringIO(LT_TYPES), sep="=", header=None, 
                       names=["lt_type", "lt"])
# ltdf = pd.read_csv(os.path.join(RESULTDIR, LT_FN), header=None, 
#                    names=["instance", "lt"])
# ltdf = ltdf.merge(lt_types, on="lt")
# ltdf

In [145]:
# print(os.path.exists(RESULTDIR))

dfs = []


for exp in EXP_NAMES:
    #resultdir = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "concurrent", exp)
    #resultdir = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "background", exp)
    #resultdir = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "new", exp)
    
    for rdir in RESULTDIRS:
        resultdir = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", rdir, exp)
        #assert os.path.exists(resultdir)
        if not os.path.exists(resultdir):
            continue
        ltdf = pd.read_csv(os.path.join(resultdir, LT_FN), header=None, 
                       names=["instance", "lt"],

                          )
        ltdf = ltdf.merge(lt_types, on="lt")

        rtfns = glob.iglob(resultdir + "/*/results/Runtime*.csv")

        for rtfn in rtfns:
            instance_name = os.path.basename(os.path.dirname(os.path.dirname(rtfn)))
            rt = pd.read_csv(rtfn, on_bad_lines="skip")
            rt["instance"] = instance_name
            rt = rt.merge(ltdf, on="instance")
            if len(rt) == 0:
                print(instance_name)
                continue
            mintime = min(rt["start_time"].values)
            rt["relative_start_time"] = rt.apply(lambda x: x["start_time"]-mintime , axis=1)
            rt["tag"] = exp

            dfs.append(rt)
    
df = pd.concat(dfs)

i-062356c12e9a13a20


In [146]:
print(df["runtime"].sum())
print(df["runtime"].describe())
print(df.keys())

784918.4128620625
count    15412.000000
mean        50.929043
std        134.201857
min          0.002880
25%          1.164498
50%          5.340502
75%         29.087115
max        909.000000
Name: runtime, dtype: float64
Index(['start_time', 'qname', 'runtime', 'exp_analyze', 'samples_type',
       'parallel_workers', 'no_index', 'drop_cache', 'col_store', 'result_dir',
       'query_dir', 'cost_model', 'materialize', 'explain', 'reps',
       'num_queries', 'timeout', 'rerun_timeouts', 'db_name', 'db_host',
       'user', 'pwd', 'port', 'instance', 'lt', 'lt_type',
       'relative_start_time', 'tag', 'dbms'],
      dtype='object')


In [147]:
df = df[df["runtime"] >= 0.0]
df = df[df["runtime"] < 900.0]

In [148]:
print(len(df.groupby(["lt_type"])["runtime"].describe(percentiles=[0.90, 0.99])))
df.groupby(["query_dir", "lt_type"])["runtime"].describe(percentiles=[0.90, 0.99]).round(2)

12


count    mean     std   min    50%  \
query_dir           lt_type                                                  
./queries/ceb-small a1_large_gp3_4g     894.0   78.83  111.53  0.22  32.33   
                    a1_large_mag_4g     103.0  208.43  246.41  0.44  99.60   
                    c5a_large_mag_4g    325.0  204.80  239.33  0.09  94.91   
                    c7g_large_mag_4g    213.0  195.63  236.39  0.29  90.31   
                    m6a_large_mag_8g    900.0   70.50  102.75  0.05  29.96   
                    r6a_large_mag_16g   900.0    7.32   20.69  0.03   2.00   
                    r7g_large_gp2_16g   900.0    9.13   25.35  0.04   2.38   
                    r7g_medium_gp2_16g  896.0   33.49   96.06  0.06   5.13   
                    t3_large_gp2_8g     897.0   34.95   86.77  0.06   7.27   
                    t3_xlarge_gp2_16g   900.0   11.80   31.15  0.05   3.70   
                    t3a_medium_gp3_4g   900.0   69.48  113.30  0.09  24.35   
                    t4g_large_mag_8g    900.0   72.36  103.70  0.09  29.08   
./queries/job       a1_large_gp3_4g     678.0   33.93   64.65  0.02   8.09   
                    a1_large_mag_4g     107.0  113.77  201.76  0.02  21.98   
                    c5a_large_mag_4g    214.0  110.02  195.21  0.00  21.14   
                    c7g_large_mag_4g    214.0  108.29  192.91  0.01  19.45   
                    m6a_large_mag_8g    628.0   32.50   68.47  0.00   5.25   
                    r6a_large_mag_16g   678.0    2.59    6.45  0.00   0.62   
                    r7g_large_gp2_16g   678.0    2.62    4.11  0.01   0.90   
                    r7g_medium_gp2_16g  677.0   13.47   51.47  0.01   1.73   
                    t3_large_gp2_8g     678.0    7.78   22.92  0.00   2.03   
                    t3_xlarge_gp2_16g   678.0    3.95    6.48  0.01   1.14   
                    t3a_medium_gp3_4g   678.0   25.77   52.01  0.01   5.72   
                    t4g_large_mag_8g    616.0   34.16   69.02  0.01   6.53   

                                           90%     99%     max  
query_dir           lt_type                                     
./queries/ceb-small a1_large_gp3_4g     229.95  516.18  764.77  
                    a1_large_mag_4g     685.20  849.56  852.10  
                    c5a_large_mag_4g    658.90  798.21  834.41  
                    c7g_large_mag_4g    609.35  845.21  897.43  
                    m6a_large_mag_8g    206.05  495.63  716.75  
                    r6a_large_mag_16g    13.52  104.49  202.21  
                    r7g_large_gp2_16g    16.66  130.73  243.86  
                    r7g_medium_gp2_16g   77.96  561.58  895.86  
                    t3_large_gp2_8g      85.25  466.62  881.77  
                    t3_xlarge_gp2_16g    24.49  172.25  402.11  
                    t3a_medium_gp3_4g   196.10  556.00  899.28  
                    t4g_large_mag_8g    217.52  491.17  544.47  
./queries/job       a1_large_gp3_4g     111.58  312.92  400.56  
                    a1_large_mag_4g     456.78  774.04  880.45  
                    c5a_large_mag_4g    429.36  727.48  892.32  
                    c7g_large_mag_4g    424.23  730.18  882.70  
                    m6a_large_mag_8g     78.87  343.72  595.28  
                    r6a_large_mag_16g     7.09   19.75  114.49  
                    r7g_large_gp2_16g     6.90   19.17   31.32  
                    r7g_medium_gp2_16g   16.67  290.02  665.56  
                    t3_large_gp2_8g      18.64   80.90  340.79  
                    t3_xlarge_gp2_16g    11.33   31.74   56.34  
                    t3a_medium_gp3_4g    84.90  275.30  335.03  
                    t4g_large_mag_8g     94.80  341.53  540.95

In [149]:
import os

def get_workload(row):
    return os.path.basename(row["query_dir"])

#     if "ceb" in row["query_dir"]:
#         return "CEB"
#     elif "stack" in row["query_dir"]:
#         return "Stack"
#     else:
#         return os.path.basename(row["query_dir"])
    
df["workload"] = df.apply(lambda x: get_workload(x) , axis=1)

In [150]:
df = df[df["runtime"] < 900.0]

# Group by 'lt_type' and 'query', then calculate variance for 'runtime'
variance_per_group = df.groupby(['lt_type', 'qname'])['runtime'].std()

# Calculate the average of these variances
average_variance = variance_per_group.mean()

print(average_variance)
variance_per_group.describe()

11.758477197870095


count    2778.000000
mean       11.758477
std        34.283761
min         0.000050
25%         0.096988
50%         0.893428
75%         6.579824
max       403.093870
Name: runtime, dtype: float64

In [151]:
#df = pd.DataFrame(data)

# Group by 'tag', 'lt_type', and 'query', then calculate standard deviation for 'runtime'
std_per_group = df.groupby(['tag', 'lt_type', 'qname'])['runtime'].std()

# For each 'tag', calculate the average of these standard deviations
average_std_per_tag = std_per_group.groupby('tag').mean()

# Converting the Series to a DataFrame for better table formatting
average_std_table = average_std_per_tag.reset_index()
average_std_table.columns = ['Tag', 'Average_Std']

average_std_table


,Tag,Average_Std
0,exp13_imdb,2.246328
1,exp1_single,9.821424
2,exp2_single,0.826920
3,exp3_single,29.792069


In [159]:
#df = pd.DataFrame(data)

# Group by 'tag', 'lt_type', and 'query', then calculate standard deviation for 'runtime'
std_per_group = df.groupby(['workload', 'lt_type', 'qname'])['runtime'].var()

# For each 'tag', calculate the average of these standard deviations
average_std_per_tag = std_per_group.groupby('workload').mean()

# Converting the Series to a DataFrame for better table formatting
average_std_table = average_std_per_tag.reset_index()
average_std_table.columns = ['Tag', 'Average_Var']

average_std_table = average_std_table.sort_values(by="Average_Var", ascending=False)
average_std_table


,Tag,Average_Var
0,ceb-small,2026.109223
1,job,417.319342


In [160]:
#df = pd.DataFrame(data)

# Group by 'tag', 'lt_type', and 'query', then calculate standard deviation for 'runtime'
std_per_group = df.groupby(['workload', 'lt_type', 'qname'])['runtime'].std()

# For each 'tag', calculate the average of these standard deviations
average_std_per_tag = std_per_group.groupby('workload').mean()

# Converting the Series to a DataFrame for better table formatting
average_std_table = average_std_per_tag.reset_index()
average_std_table.columns = ['Tag', 'Average_Std']

average_std_table = average_std_table.sort_values(by="Average_Std", ascending=False)
average_std_table


,Tag,Average_Std
0,ceb-small,16.518749
1,job,5.776235


In [161]:
import pandas as pd

# Assuming df is your DataFrame and is already defined

# First group by 'workload', 'lt_type', and 'qname'
grouped = df.groupby(['workload', 'lt_type', 'qname'])

# Function to calculate the average std for each group
def average_std(group):
    # Group by 'tag' within the larger group and calculate std
    std_per_tag = group.groupby('tag')['runtime'].std()
    
    # Return the average of these stds
    return std_per_tag.mean()

# Apply the function to each group and reset index
average_std_per_workload = grouped.apply(average_std).reset_index(name='Average_Std')
average_std_per_tag = average_std_per_workload.groupby('workload').mean()

# Sort values by 'Average_Std'
average_std_per_tag = average_std_per_tag.sort_values(by='Average_Std', ascending=False)

average_std_per_tag

/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_5140/3017867846.py:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_std_per_tag = average_std_per_workload.groupby('workload').mean()


,Average_Std
workload,
ceb-small,9.007146
job,3.964239


In [162]:
df.groupby("workload")["start_time"].count()

workload
ceb-small    8728
job          6524
Name: start_time, dtype: int64

In [163]:
df.groupby("workload")["qname"].nunique()

workload
ceb-small    150
job          113
Name: qname, dtype: int64

In [164]:
print(len(set(df["qname"])))

263


In [165]:
import pandas as pd

# Assuming df is your DataFrame and is already defined

# First group by 'workload', 'lt_type', and 'qname'
grouped = df.groupby(['workload', 'lt_type', 'qname'])

# Function to calculate the average std for each group
def average_std(group):
    # Group by 'tag' within the larger group and calculate std
    std_per_tag = group.groupby('qname')['runtime'].std()
    
    # Return the average of these stds
    return std_per_tag.mean()

# Apply the function to each group and reset index
average_std_per_workload = grouped.apply(average_std).reset_index(name='Average_Std')
average_std_per_workload
average_std_per_tag = average_std_per_workload.groupby(['workload', 'lt_type']).mean()

# # Sort values by 'Average_Std'
#average_std_per_tag = average_std_per_tag.sort_values(by='Average_Std', ascending=False)

average_std_per_tag

/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_5140/3282528414.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_std_per_tag = average_std_per_workload.groupby(['workload', 'lt_type']).mean()


Average_Std
workload  lt_type                        
ceb-small a1_large_gp3_4g       13.925307
          a1_large_mag_4g             NaN
          c5a_large_mag_4g      11.751588
          c7g_large_mag_4g       4.856583
          m6a_large_mag_8g      15.944500
          r6a_large_mag_16g      1.705704
          r7g_large_gp2_16g      0.593206
          r7g_medium_gp2_16g    50.543941
          t3_large_gp2_8g       43.154019
          t3_xlarge_gp2_16g      2.188310
          t3a_medium_gp3_4g     17.309245
          t4g_large_mag_8g      13.533009
job       a1_large_gp3_4g        3.687079
          a1_large_mag_4g             NaN
          c5a_large_mag_4g       6.058476
          c7g_large_mag_4g       3.582501
          m6a_large_mag_8g       5.770621
          r6a_large_mag_16g      1.163498
          r7g_large_gp2_16g      0.096358
          r7g_medium_gp2_16g    22.532289
          t3_large_gp2_8g        5.108367
          t3_xlarge_gp2_16g      0.565581
          t3a_medium_gp3_4g      6.524922
          t4g_large_mag_8g       8.347395